In [1]:
import pandas as pd

pets_100_df = pd.read_json('Pet_Supplies_100.json', lines=True)

In [2]:
pets_100_df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A14CK12J7C7JRK,1223000893,Consumer in NorCal,"[0, 0]",I purchased the Trilogy with hoping my two cat...,3,Nice Distraction for my cats for about 15 minutes,1294790400,"01 12, 2011"
1,A39QHP5WLON5HV,1223000893,Melodee Placial,"[0, 0]",There are usually one or more of my cats watch...,5,Entertaining for my cats,1379116800,"09 14, 2013"
2,A2CR37UY3VR7BN,1223000893,Michelle Ashbery,"[0, 0]",I bought the triliogy and have tested out all ...,4,Entertaining,1355875200,"12 19, 2012"
3,A2A4COGL9VW2HY,1223000893,Michelle P,"[2, 2]",My female kitty could care less about these vi...,4,Happy to have them,1305158400,"05 12, 2011"
4,A2UBQA85NIGLHA,1223000893,"Tim Isenhour ""Timbo""","[6, 7]","If I had gotten just volume two, I would have ...",3,You really only need vol 2,1330905600,"03 5, 2012"


In [3]:
pd.value_counts(pets_100_df['overall'])

5    29
4    26
1    20
3    16
2     9
Name: overall, dtype: int64

In [4]:
class Review:
    def __init__(self, text, score):
        self.text = text
        self.score = score
        self.sentiment = 'NEGATIVE' if score <=2 else 'NEUTRAL' if score <= 3 else 'POSITIVE'

In [9]:
import contractions
import string
import re

def clean_text(text):
    text = contractions.fix(text)
    text = text.lower()
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub(r'\w*\d\w*', '', text)
    stopwords = [word.strip() for word in open('stopwords_en.txt', 'r')]
    text = ' '.join([word for word in text.split() if word not in stopwords])
    return text

In [16]:
import json

reviews = []
with open('Pet_Supplies_10000.json', 'r') as in_file:
    for line in in_file:
        data = json.loads(line)
        text = data['reviewText']
        score = data['overall']
        text = clean_text(text)
        reviews.append(Review(text, score))

In [21]:
reviews[201].sentiment

'POSITIVE'

In [22]:
len(reviews)

10000

In [23]:
from sklearn.model_selection import train_test_split
training_data, test_data = train_test_split(reviews, test_size=0.33, random_state=42)

In [25]:
len(test_data)

3300

In [26]:
train_x = [review.text for review in training_data]
train_y = [review.sentiment for review in training_data]

test_x = [review.text for review in test_data]
test_y = [review.sentiment for review in test_data]

In [27]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
train_x_vectors = vectorizer.fit_transform(train_x)

In [28]:
from sklearn.svm import SVC

clf_svm = SVC(kernel='linear')
clf_svm.fit(train_x_vectors, train_y)

SVC(kernel='linear')

In [29]:
test_x_vectors = vectorizer.transform(test_x)

print('The result is:', clf_svm.score(test_x_vectors, test_y))

The result is: 0.7833333333333333


In [34]:
print(clf_svm.predict(test_x_vectors[2]))
print(test_x[2])
print(test_y[2])


['NEGATIVE']
seems work far cats drinking water unfortunately none four cats like food much dry foods tried
NEUTRAL


In [35]:
text = "Great comb for getting loose hair out!"
text_cleaned = clean_text(text)

In [36]:
input_vector = vectorizer.transform([text_cleaned])

ValueError: Iterable over raw text documents expected, string object received.